In [ ]:
from twinews import config as twinewsConf
twinewsConf.mongoLocation = 'octods'

In [ ]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [ ]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from datastructuretools.cache import *
from twinews.utils import *
from twinews.evaluation.utils import *
from twinews.models.ranking import *

In [ ]:
from dataviztools import bokehutils
from bokeh.plotting import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
output_notebook()

In [ ]:
import importlib
importlib.reload(bokehutils)

In [ ]:
logger = Logger()

In [ ]:
# NOTE IMPORTANTE
# Si on a un "CorruptGridFile: no chunk #0" après un restore, faire :
# pruneRankings('combin-d0fd0')

# Printing stats on rankings

In [ ]:
twinewsRankings = getTwinewsRankings()
splitVersion = 1
models = set()
for row in twinewsRankings.collection.find({"meta.splitVersion": splitVersion, 'meta.maxUsers': None}):
    if row['meta']['model'] != 'combin':
        models.add(row['meta']['model'])
models.remove('worst')
models.remove('ideal')
models.remove('random')
keys = set()
for model in models:
    keys.add(twinewsRankings.collection.find_one({'meta.model': model,
                                                  "meta.splitVersion": splitVersion,
                                                  'meta.maxUsers': None})['id'])
# keys.add("combin-8f846")
# keys.add("combin-efad4")
keys = sorted(list(keys))
print(keys)

In [ ]:
for key in twinewsRankings.keys():
    meta = twinewsRankings.getMeta(key)
    if 'models' in meta and 'doc2vec-e013a' in meta['models'] and 'dbert-ft-7847a' in meta['models']:
        print(meta)

 * bm25-933f7
 * tfidf-4a1dc
 * bm25-933f7 + dbert-ft-7847a = combin-8f846
 * dbert-ft-7847a + tfidf-4a1dc = combin-6ecf3
 * bm25-933f7 + doc2vec-e013a = combin-6fad8
 * dbert-ft-7847a + doc2vec-e013a = combin-d0fd0

In [ ]:
# twinewsRankings.collection.find_one({'id': 'combin-8f846', 'length': {'$gt': 100}})
# twinewsRankings.getMeta('combin-7baab')

In [ ]:
getTwinewsScores().findOne({'metric': 'ndcg', 'id': 'combin-d0fd0'})

In [ ]:
rankingsCache = dict()

In [ ]:
from twinews.evaluation import metrics
metricFunct = metrics.ndcg

In [ ]:
twinewsScores = getTwinewsScores()
evalData = getEvalData(splitVersion, logger=logger, maxExtraNews=0)
candidates = evalData['candidates']

In [ ]:
import numpy as np
import scipy.stats
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    # return m, m-h, m+h, h
    return h

In [ ]:
for model in \
[
    # 'bm25-933f7',
    # 'combin-8f846',
    'combin-6fad8',
    'combin-d0fd0'
]:
    log("Getting rankings of " + model, logger)
    if model in rankingsCache:
        rankings = rankingsCache[model]
    else:
        rankings = twinewsRankings[model]
        rankingsCache[model] = rankings
    checkRankings(rankings, candidates, maxUsers=None)
    # We convert all in a list of rel vectors:
    rels = []
    for userId in rankings:
        for ranking in rankings[userId]:
            if isinstance(ranking[0], tuple):
                ranking = [e[0] for e in ranking]
            rel = rankingToRelevanceVector(ranking, set(evalData['testUsers'][userId].keys()))
            rels.append(rel)
    # We compute all scores:
    log("Computing scores of " + model, logger)
    scores = []
    for rel in rels:
        scores.append(metricFunct(rel))
    bp(scores, logger)
    log("Scores count: " + str(len(scores)), logger)
    # We mean all scrores:
    score = float(np.mean(scores))
    log("Mean score is " + str(score), logger)
    # Confidence interval
    log("mean_confidence_interval of scores:", logger)
    log("\t-->" + str(mean_confidence_interval(scores)), logger)
    # We split:
    splits = split(scores, 30)
    splits = splits[:-1]
    log("Splits lengths:", logger)
    bp([len(e) for e in splits], 4, logger=logger)
    means = []
    for current in splits:
        means.append(float(np.mean(current)))
    log("Means:", logger)
    bp(means, 4, logger=logger)
    log("mean_confidence_interval of means:", logger)
    log("\t-->" + str(mean_confidence_interval(means)), logger)

 * 0.0034250413659661937
 * 0.003373019267614974
 * 0.0033745450496996477
 * 0.0033251230353648917